## Example for extending Mondrian Trees

In [1]:
import Base.copy
using Distributions
using MLBase
using Plots
include("./Mondrian_Forest_Classifier.jl"); 
include("Mondrian_extention.jl")

expand_forest!

This notebook explains how to extend Mondrian Trees and Mondrian Forests. Using an online algorithm, we can incorporate data points one by one.

### Functions

All functions used in this section can be found in the files "Mondrian_extention.jl" and "Mondrian_extention_utils.jl". 

In [2]:
?Extend_Mondrian_Tree!

search:



`function Extend_Mondrian_Tree!(T::Mondrian_Tree,λ::Float64,X::Array{Float64} where N,Y::Int64)`

This function extends an already existing Mondrian Tree by ONE new datapoint that gets incorperated in the tree. 

`Input`: Mondrian Tree T (abstract type Mondrian_Tree), Lifetime parameter λ (Float64), 1dim Array of Features Array X (Array of Float64), classlabel Y (Int64)

`Output`: Mondrian Tree

`Files needed to run this function`: "Mondrian_Forest_Classifier.jl", "Mondrian_extention.jl"

This function calls the function Extend_Mondrian_Block.

The usage of the function "expand!" is recommended to expand Mondrian Trees as it has a nicer user interface.


In [ ]:
?Extend_Mondrian_Tree!

In [ ]:
?expand!

In [ ]:
?expand_forest!

#### 1) Prepare some data

We generate two sets of fakedata (features + classes). We use the first one to train on and the second one to extend on.

In [2]:
using MLBase
using Plots

function Fakedata3Classif(N,d,N_test=0)
    X = randn((N,d))
    param1 = randn(d)
    param2 = randn(d)
    Y = ( sum(X*param1,2) .> mean(sum(X*param2,2)) )
    Y=convert(Array{Int},Y)
    for i in 1:size(X,1)
        if X[i,2] > 1.5
            Y[i] = 2
        end
    end
    if (N_test > 0)
        x = randn((N_test,d))
        y = ( sum(x*param1,2) .> mean(sum(x*param2,2)) )
        y=convert(Array{Int},y)
        for i in 1:size(x,1)
            if x[i,2] > 1.5
                y[i] = 2
            end
        end
        return X,Y,x,y
    end
    return X,Y
end

Fakedata3Classif (generic function with 2 methods)

In [ ]:
X, Y, x, y = Fakedata3Classif(5000,2,5000);  #X,Y: train set ; x,y: extention data
Y = Y.+1;
y = y.+1;

In [ ]:
if size(X,2)==2
    plot()
    for class in unique(Y)
        scatter!(X[Y.==class,1],X[Y.==class,2],label="$class")
    end
    plot!()
end

### 1) Extend a single tree

To extend a single tree, we can just use the function expand!. We train a Mondrian Tree Classifier on the dataset X,Y and then extend it on the dataset x,y which we created above. We compute the training accuracy of the extended tree. For comparison, we then train a Mondrian Tree Classifier on the merges dataset X,x Y,y (it was then trained on all of the data the extended tree was trained on, without using extentions) and compute the training accuracy as well.  

In [ ]:
T = Mondrian_Tree_Classifier()
train!(T,X,Y,1e9);  #Sample tree on training dataset

In [ ]:
expand!(T,x,y,1e9);  #Expand the tree on the extention dataset

In [ ]:
#merge training and extention data to get the total data the tree was trained on

All_data=vcat(X,x);             
all_labels = vcat(Y,y);

In [ ]:
#let the extended tree predict the training data, compute training accuracy
pred=predict!(T,All_data);     
println("Train Accuracy of extended tree")
println(correctrate(all_labels,convert(Array{Int,1},pred)))

In [ ]:
#compute a tree on the merged dataset (classic, no extention)
T_full = Mondrian_Tree_Classifier()
train!(T_full,All_data,all_labels,1e9);

In [ ]:
#let the extended tree predict the training data, compute training accuracy

println("Train Accuracy tree trained on whole dataset")
pred=predict!(T_full,All_data)
println(correctrate(all_labels,convert(Array{Int,1},pred)))

We see that the training accuracy is comparable weather we extend the tree or weather we sample the tree on the whole dataset immediatly. This proves that the extention implementation works.

### 2) Extend Mondrian Forests

Now we extend a Mondrian Forest classifier that was pretrained on the  training same dataset (X,Y) as above. We again extend the classifier and then train a new classifier on the whole datset for comparison.

In [ ]:
MF = Mondrian_Forest_Classifier(50)  #initialise Mondrian Forest Classifier with 50 Trees

In [ ]:
train!(MF, X, Y, 1e9);  #Train the classifier on the same dataset as before

In [ ]:
expand_forest!(MF,x,y,1e9);  #expand the forest on the dataset x,y

In [ ]:
#compute train accuracy for extended forest

pred=predict!(MF, All_data);
println("Train Accuracy")
println(correctrate(all_labels,convert(Array{Int,1},pred)))

In [ ]:
MF_control = Mondrian_Forest_Classifier(50)   #initialise control classifier with 50 Trees
train!(MF_control, All_data, all_labels, 1e9);  # train control classifier on merged dataset

In [ ]:
#compute train accuracy for classic forest

pred = predict!(MF_control,All_data);
println("Train Accuracy")
println(correctrate(all_labels,convert(Array{Int,1},pred)))

#### Some Testing and Benchmarking

How fast is the extention? 

In [1089]:
X, Y,x ,y = Fakedata3Classif(5000,4,5000); #X,Y: train set ; x,y: extention data
Y = Y.+1;
y = y.+1;

In [1090]:
println(size(X))

(5000, 4)


In [781]:
MF=Mondrian_Forest_Classifier(100)
Acc=zeros(150);
Time=[];

In [ ]:
counter=1
train!(MF,X,Y,1e9);
pred=predict!(MF,X)
Acc[counter]=correctrate(Y,convert(Array{Int,1},pred))

In [ ]:
expand_forest!(MF,x,y,1e9);
pred=predict!(MF,vcat(X,x))
Acc[counter]=correctrate(vcat(Y,y),convert(Array{Int,1},pred))

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

### batch training

##### Single tree

In [3]:
X, Y,x ,y = Fakedata3Classif(100,2,100); #X,Y: train set ; x,y: extention data
Y = Y.+1;
y = y.+1;

In [4]:
T = Mondrian_Tree_Classifier()
train!(T,X[1:20,:],Y[1:20],1e9)  #Sample tree on training dataset

Mondrian Tree Classifier:
    Mondrian Tree with 

7 leaves

In [1436]:
j_print((T.Tree).root)

[8, 10, 2]
[8, 8, 2]
[0, 2, 0]
node done
[8, 8, 2]
[0, 4, 0]
[8, 4, 2]
node done
[8, 4, 2]
[1, 0, 1]
[7, 4, 1]
node done
[1, 0, 1]
[1, 0, 0]
[0, 0, 1]
node done
[7, 4, 1]
[7, 4, 0]
[0, 0, 1]
node done
[7, 4, 0]
[6, 1, 0]
[1, 3, 0]
node done
[1, 3, 0]
[0, 2, 0]
[1, 1, 0]
node done
[1, 1, 0]
[1, 0, 0]
[0, 1, 0]
node done


In [1437]:
ϵ =  (T.Tree).root
get(ϵ).c
get(ϵ).node_type
leaf_check(ϵ)


[0, 4, 0]
[1, 0, 0]
[0, 0, 1]
[6, 1, 0]
[0, 2, 0]
[1, 0, 0]
[0, 1, 0]
[0, 0, 1]
[0, 2, 0]


In [1438]:
pred=predict!(T,x)
correctrate(y,convert(Array{Int,1},pred))

0.8

In [1439]:
expand!(T,X[21:40,:],Y[21:40],1e9)

new extention
[8, 10, 2]
2
2.1
[8, 8, 2]
2
2.1
[0, 4, 0]
[8, 11, 2]
[8, 9, 2]
[0, 2, 0]
node done
[8, 9, 2]
[0, 5, 0]
[8, 4, 2]
node done
[8, 4, 2]
[1, 0, 1]
[7, 4, 1]
node done
[1, 0, 1]
[1, 0, 0]
[0, 0, 1]
node done
[7, 4, 1]
[7, 4, 0]
[0, 0, 1]
node done
[7, 4, 0]
[6, 1, 0]
[1, 3, 0]
node done
[1, 3, 0]
[0, 2, 0]
[1, 1, 0]
node done
[1, 1, 0]
[1, 0, 0]
[0, 1, 0]
node done
new extention
[8, 11, 2]
2
2.1
[8, 9, 2]
2
2.2
[8, 4, 2]
2
2.2
[7, 4, 1]
2
2.1
[7, 4, 0]
2
2.2
[1, 3, 0]
2
2.1
[0, 2, 0]
[8, 12, 2]
[8, 10, 2]
[0, 2, 0]
node done
[8, 10, 2]
[0, 5, 0]
[8, 5, 2]
node done
[8, 5, 2]
[1, 0, 1]
[7, 5, 1]
node done
[1, 0, 1]
[1, 0, 0]
[0, 0, 1]
node done
[7, 5, 1]
[7, 5, 0]
[0, 0, 1]
node done
[7, 5, 0]
[6, 1, 0]
[1, 4, 0]
node done
[1, 4, 0]
[0, 3, 0]
[1, 1, 0]
node done
[1, 1, 0]
[1, 0, 0]
[0, 1, 0]
node done
new extention
[8, 12, 2]
2
2.1
[8, 10, 2]
2
2.1
[0, 5, 0]
[8, 13, 2]
[8, 11, 2]
[0, 2, 0]
node done
[8, 11, 2]
[0, 6, 0]
[8, 5, 2]
node done
[8, 5, 2]
[1, 0, 1]
[7, 5, 1]
node do

Mondrian Tree Classifier:
    Mondrian Tree with 9 leaves

In [1440]:
pred=predict!(T,X)
correctrate(Y,convert(Array{Int,1},pred))

0.89

In [1441]:
j_print((T.Tree).root)

[13, 21, 3]
[13, 18, 3]
[0, 3, 0]
node done
[13, 18, 3]
[0, 8, 0]
[13, 10, 3]
node done
[13, 10, 3]
[4, 0, 1]
[9, 10, 2]
node done
[4, 0, 1]
[4, 0, 0]
[0, 0, 1]
node done
[9, 10, 2]
[9, 10, 0]
[0, 0, 2]
node done
[9, 10, 0]
[6, 1, 0]
[3, 9, 0]
node done
[3, 9, 0]
[0, 4, 0]
[3, 5, 0]
node done
[3, 5, 0]
[3, 0, 0]
[0, 5, 0]
node done


In [1442]:
ϵ =  (T.Tree).root
leaf_check(ϵ)

[0, 8, 0]
[4, 0, 0]
[0, 0, 1]
[6, 1, 0]
[0, 4, 0]
[3, 0, 0]
[0, 5, 0]
[0, 0, 2]
[0, 3, 0]


In [1443]:
expand!(T,X[41:60,:],Y[41:60],1e9)

new extention
[13, 21, 3]
2
2.1
[13, 18, 3]
2
2.2
[13, 10, 3]
2
2.1
[4, 0, 1]
2
2.1
[4, 0, 0]
[14, 21, 3]
[14, 18, 3]
[0, 3, 0]
node done
[14, 18, 3]
[0, 8, 0]
[14, 10, 3]
node done
[14, 10, 3]
[5, 0, 1]
[9, 10, 2]
node done
[5, 0, 1]
[5, 0, 0]
[0, 0, 1]
node done
[9, 10, 2]
[9, 10, 0]
[0, 0, 2]
node done
[9, 10, 0]
[6, 1, 0]
[3, 9, 0]
node done
[3, 9, 0]
[0, 4, 0]
[3, 5, 0]
node done
[3, 5, 0]
[3, 0, 0]
[0, 5, 0]
node done
new extention
[14, 21, 3]
2
2.1
[14, 18, 3]
2
2.2
[14, 10, 3]
2
2.2
[9, 10, 2]
2
2.1
[9, 10, 0]
2
2.2
[3, 9, 0]
2
2.2
[3, 5, 0]
2
2.2
[0, 5, 0]
[14, 22, 3]
[14, 19, 3]
[0, 3, 0]
node done
[14, 19, 3]
[0, 8, 0]
[14, 11, 3]
node done
[14, 11, 3]
[5, 0, 1]
[9, 11, 2]
node done
[5, 0, 1]
[5, 0, 0]
[0, 0, 1]
node done
[9, 11, 2]
[9, 11, 0]
[0, 0, 2]
node done
[9, 11, 0]
[6, 1, 0]
[3, 10, 0]
node done
[3, 10, 0]
[0, 4, 0]
[3, 6, 0]
node done
[3, 6, 0]
[3, 0, 0]
[0, 6, 0]
node done
new extention
[14, 22, 3]
2
2.1
[14, 19, 3]
2
2.2
[14, 11, 3]
2
2.2
[9, 11, 2]
2
2.1
[9, 11,

Mondrian Tree Classifier:
    Mondrian Tree with 10 leaves

In [1444]:
pred=predict!(T,X)
correctrate(Y,convert(Array{Int,1},pred))

0.89

In [1445]:
ϵ =  (T.Tree).root
leaf_check(ϵ)

[0, 15, 0]
[7, 0, 0]
[0, 0, 1]
[6, 1, 0]
[0, 6, 0]
[3, 0, 0]
[0, 8, 0]
[0, 0, 5]
[0, 3, 0]


In [1446]:
expand!(T,X[61:80,:],Y[61:80],1e9)

new extention
[16, 33, 6]
2
2.1
[16, 30, 6]
2
2.2
[16, 15, 6]
2
2.1
[7, 0, 1]
2
2.1
[7, 0, 0]
[17, 33, 6]
[17, 30, 6]
[0, 3, 0]
node done
[17, 30, 6]
[0, 15, 0]
[17, 15, 6]
node done
[17, 15, 6]
[8, 0, 1]
[9, 15, 5]
node done
[8, 0, 1]
[8, 0, 0]
[0, 0, 1]
node done
[9, 15, 5]
[9, 15, 0]
[0, 0, 5]
node done
[9, 15, 0]
[6, 1, 0]
[3, 14, 0]
node done
[3, 14, 0]
[0, 6, 0]
[3, 8, 0]
node done
[3, 8, 0]
[3, 0, 0]
[0, 8, 0]
node done
new extention
[17, 33, 6]
2
2.1
[17, 30, 6]
2
2.2
[17, 15, 6]
2
2.2
[9, 15, 5]
2
2.1
[9, 15, 0]
2
2.1
[6, 1, 0]
2
[17, 33, 6]
[17, 30, 6]
[0, 3, 0]
node done
[17, 30, 6]
[0, 15, 0]
[17, 15, 6]
node done
[17, 15, 6]
[8, 0, 1]
[9, 15, 5]
node done
[8, 0, 1]
[8, 0, 0]
[0, 0, 1]
node done
[9, 15, 5]
[9, 15, 0]
[0, 0, 5]
node done
[9, 15, 0]
[6, 1, 0]
[3, 14, 0]
node done
[3, 14, 0]
[0, 6, 0]
[3, 8, 0]
node done
[3, 8, 0]
[3, 0, 0]
[0, 8, 0]
node done
new extention
[17, 33, 6]
2
2.1
[17, 30, 6]
2
2.1
[0, 15, 0]
[17, 34, 6]
[17, 31, 6]
[0, 3, 0]
node done
[17, 31, 6]
[

Mondrian Tree Classifier:
    Mondrian Tree with 10 leaves

In [1447]:
pred=predict!(T,X)
correctrate(Y,convert(Array{Int,1},pred))

0.89

True Tree

In [1448]:
T=Mondrian_Tree_Classifier()
train!(T,X,Y,1e9)

PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED


Mondrian Tree Classifier:
    Mondrian Tree with 18 leaves

In [1449]:
ϵ =  (T.Tree).root
a = get(ϵ).left
 get(get(a).parent).c[2]
# get(get(j_wave).parent).c[Y]

54

In [1450]:
pred=predict!(T,X)
correctrate(Y,convert(Array{Int,1},pred))

0.86

In [1455]:
T=Mondrian_Tree_Classifier()
Acc=[];
Time=[];

In [1456]:
counter=1
train!(T,X[1:20,:],Y[1:20],1e9);
pred=predict!(T,X)
append!(Acc,correctrate(Y,convert(Array{Int,1},pred)))

PAUSED
PAUSED


1-element Array{Any,1}:
 0.7

In [1457]:
i=21
while i<100
    tic()
    expand!(T,X[i:i+19,:],Y[i:i+19,:],1e9)
    println(T)
    t=toq()
    println("The time is")
    println(t)
    push!(Time,t)
    pred=predict!(T,X)
    append!(Acc,correctrate(Y,convert(Array{Int,1},pred)))
    i=i+20
    println("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
end

new extention
[8, 10, 2]
2
2.1
[0, 7, 0]
[8, 11, 2]
[0, 8, 0]
[8, 3, 2]
node done
[8, 3, 2]
[8, 2, 2]
[0, 1, 0]
node done
[8, 2, 2]
[1, 0, 1]
[7, 2, 1]
node done
new extention
[8, 11, 2]
2
2.2
[8, 3, 2]
2
2.1
[8, 2, 2]
2
2.2
[7, 2, 1]
2
[8, 11, 2]
[0, 8, 0]
[8, 3, 2]
node done
[8, 3, 2]
[8, 2, 2]
[0, 1, 0]
node done
[8, 2, 2]
[1, 0, 1]
[7, 2, 1]
node done
new extention
[8, 11, 2]
2
2.1
[0, 8, 0]
[8, 12, 2]
[0, 9, 0]
[8, 3, 2]
node done
[8, 3, 2]
[8, 2, 2]
[0, 1, 0]
node done
[8, 2, 2]
[1, 0, 1]
[7, 2, 1]
node done
new extention
[8, 12, 2]
2
2.1
[0, 9, 0]
PAUSED
[8, 12, 2]
[0, 9, 0]
[8, 3, 2]
node done
[8, 3, 2]
[8, 2, 2]
[0, 1, 0]
node done
[8, 2, 2]
[1, 0, 1]
[7, 2, 1]
node done
new extention
[8, 12, 2]
2
2.2
[8, 3, 2]
2
2.1
[8, 2, 2]
2
2.1
[1, 0, 1]
1
Nullable{Mondrian_Node}(Mondrian_Node(Mondrian_Node(Mondrian_Node(#NULL, Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([-1.86584 2.44321; -2.1829

Nullable{Mondrian_Node}(Mondrian_Node(Mondrian_Node(Mondrian_Node(#NULL, Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([-1.86584 2.44321; -2.1829 0.00753811], 2), [0, 1, 0], [0, 12, 0], [0.0, 1.0, 0.0], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(#= circular reference @-2 =#), 0.0994242, Bool[false, false, true], 2, 0.00753811, Axis_Aligned_Box([-2.01681 2.44321; -2.1829 2.44609], 2), [1, 1, 1], [11, 15, 2], [0.4, 0.5, 0.1], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(#= circular reference @-2 =#), Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([0.803067 2.44321; 0.00753811 2.44609], 2), [0, 1, 0], [0, 1, 0], [0.0, 1.0, 0.0], Int64[], 0.0, 0.0, 0.0), 0.288938, Bool[true, false, false], 1, 0.803067, Axis_Aligned_Box([-2.01681 2.44321; 0.00753811 2.44609], 2), [1, 1, 1], [11, 3, 2], [0.615732, 0.231638, 0.15263], Int64[

Nullable{Mondrian_Node}(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(#NULL, Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([-1.86584 2.44321; -2.1829 0.00753811], 2), [0, 1, 0], [0, 16, 0], [0.0, 1.0, 0.0], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(#= circular reference @-2 =#), 0.0994242, Bool[false, false, true], 2, 0.00753811, Axis_Aligned_Box([-2.01681 2.44321; -2.1829 2.44609], 2), [1, 1, 1], [12, 24, 2], [0.342857, 0.6, 0.0571429], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(#= circular reference @-2 =#), Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([0.803067 2.44321; 0.00753811 2.44609], 2), [0, 1, 0], [0, 2, 0], [0.0, 1.0, 0.0], Int64[], 0.0, 0.0, 0.0), 0.288938, Bool[true, false, false], 1, 0.803067, Axis_Aligned_Box([-2.01681 2.44321; 0.00753811 2.44609

[0, 4, 0]
node done
[7, 0, 1]
[4, 0, 1]
[3, 0, 0]
node done
[4, 0, 1]
[1, 0, 0]
[3, 0, 1]
node done
[3, 0, 1]
[1, 0, 1]
[2, 0, 1]
node done
[2, 0, 1]
[1, 0, 0]
[1, 0, 1]
node done
new extention
[14, 31, 2]
2
2.2
[14, 10, 2]
2
2.2
[0, 4, 0]
PAUSED
[14, 31, 2]
[0, 21, 0]
[14, 10, 2]
node done
[14, 10, 2]
[14, 6, 2]
[0, 4, 0]
node done
[14, 6, 2]
[1, 0, 1]
[7, 4, 1]
node done
[7, 4, 1]
[7, 0, 1]
[0, 4, 0]
node done
[7, 0, 1]
[4, 0, 1]
[3, 0, 0]
node done
[4, 0, 1]
[1, 0, 0]
[3, 0, 1]
node done
[3, 0, 1]
[1, 0, 1]
[2, 0, 1]
node done
[2, 0, 1]
[1, 0, 0]
[1, 0, 1]
node done
new extention
[14, 31, 2]
2
2.1
[0, 21, 0]
[14, 32, 2]
[0, 22, 0]
[14, 10, 2]
node done
[14, 10, 2]
[14, 6, 2]
[0, 4, 0]
node done
[14, 6, 2]
[1, 0, 1]
[7, 4, 1]
node done
[7, 4, 1]
[7, 0, 1]
[0, 4, 0]
node done
[7, 0, 1]
[4, 0, 1]
[3, 0, 0]
node done
[4, 0, 1]
[1, 0, 0]
[3, 0, 1]
node done
[3, 0, 1]
[1, 0, 1]
[2, 0, 1]
node done
[2, 0, 1]
[1, 0, 0]
[1, 0, 1]
node done
new extention
[14, 32, 2]
2
2.2
[14, 10, 2]
2
2.1
[1

Nullable{Mondrian_Node}(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(#NULL, Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([-1.86584 2.44321; -2.1829 0.00753811], 2), [0, 1, 0], [0, 27, 0], [0.0, 1.0, 0.0], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(#= circular reference @-2 =#), 0.0994242, Bool[false, false, true], 2, 0.00753811, Axis_Aligned_Box([-2.01681 2.44321; -2.1829 2.44609], 2), [1, 1, 1], [15, 39, 2], [0.291667, 0.666667, 0.0416667], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(#= circular reference @-2 =#), Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([0.803067 2.44321; 0.00753811 2.44609], 2), [0, 1, 0], [0, 6, 0], [0.0, 1.0, 0.0], Int64[], 0.0, 0.0, 0.0), 0.288938, Bool[true, false, false], 1, 0.803067, Axis_Aligned_Box([-2.01681 2.44321;

Nullable{Mondrian_Node}(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(Mondrian_Node(#NULL, Mondrian_Node(#= circular reference @-2 =#), Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([-1.21555 2.65579; -2.1829 2.44609], 2), [0, 1, 0], [0, 1, 0], [0.0, 0.0, 0.0], Int64[], 0.0, 0.0, 0.0), 0.0870439, Bool[false, false, true], 1, 2.65579, Axis_Aligned_Box([-1.21555 2.65579; -2.1829 2.44609], 2), [1, 1, 1], [17, 41, 2], [0.0, 0.0, 0.0], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(Mondrian_Node(#= circular reference @-2 =#), #NULL, #NULL, 1.0e9, Bool[false, true, false], #NULL, #NULL, Axis_Aligned_Box([-1.86584 2.44321; -2.1829 0.00753811], 2), [0, 1, 0], [0, 28, 0], [0.0, 1.0, 0.0], Int64[], 0.0, 0.0, 0.0), Mondrian_Node(#= circular reference @-2 =#), 0.0994242, Bool[true, false, false], 2, 0.00753811, Axis_Aligned_Box([-1.21555

[18, 42, 3]
2
2.2
[18, 13, 3]
2
2.1
[18, 6, 3]
2
2.2
[11, 4, 2]
2
2.1
[11, 0, 2]
2
2.2
[5, 0, 0]
[19, 42, 3]
[0, 29, 0]
[19, 13, 3]
node done
[19, 13, 3]
[19, 6, 3]
[0, 7, 0]
node done
[19, 6, 3]
[1, 0, 1]
[12, 4, 2]
node done
[12, 4, 2]
[12, 0, 2]
[0, 4, 0]
node done
[12, 0, 2]
[6, 0, 2]
[6, 0, 0]
node done
[6, 0, 2]
[1, 0, 0]
[5, 0, 2]
node done
[5, 0, 2]
[1, 0, 1]
[4, 0, 2]
node done
[4, 0, 2]
[1, 0, 0]
[3, 0, 2]
node done
[3, 0, 2]
[1, 0, 0]
[2, 0, 2]
node done
[2, 0, 2]
[1, 0, 0]
[1, 0, 2]
node done
[1, 0, 2]
[0, 0, 1]
[1, 0, 1]
node done
new extention
[19, 42, 3]
2
2.2
[19, 13, 3]
2
2.1
[19, 6, 3]
2
2.2
[12, 4, 2]
2
2.1
[12, 0, 2]
2
2.1
[6, 0, 2]
2
2.2
[5, 0, 2]
2
2.2
[4, 0, 2]
2
2.2
[3, 0, 2]
2
2.2
[2, 0, 2]
2
2.2
[1, 0, 2]
2
2.2
[1, 0, 1]
2
[19, 42, 3]
[0, 29, 0]
[19, 13, 3]
node done
[19, 13, 3]
[19, 6, 3]
[0, 7, 0]
node done
[19, 6, 3]
[1, 0, 1]
[12, 4, 2]
node done
[12, 4, 2]
[12, 0, 2]
[0, 4, 0]
node done
[12, 0, 2]
[6, 0, 2]
[6, 0, 0]
node done
[6, 0, 2]
[1, 0, 0]
[5, 0, 2

In [1458]:
Acc

5-element Array{Any,1}:
 0.7 
 0.74
 0.74
 0.74
 0.74

In [858]:
j_print((T.Tree).root)

[46, 22, 7]
[38, 22, 0]
[38, 22, 0]
node done
[38, 22, 0]
[26, 10, 0]
[0, 5, 0]
node done
[26, 10, 0]
[0, 2, 0]
[26, 8, 0]
node done
[26, 8, 0]
[26, 0, 0]
[0, 8, 0]
node done
[38, 22, 0]
[0, 1, 0]
[38, 22, 0]
node done
[38, 22, 0]
[26, 10, 0]
[0, 5, 0]
node done
[26, 10, 0]
[0, 2, 0]
[26, 8, 0]
node done
[26, 8, 0]
[26, 0, 0]
[0, 8, 0]
node done


XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [1460]:
X, Y,x ,y = Fakedata3Classif(1500,2,1500); #X,Y: train set ; x,y: extention data
Y = Y.+1;
y = y.+1;

In [1461]:
T=Mondrian_Tree_Classifier()
Acc=zeros(10);
Time=[];

In [1462]:
counter=1
train!(T,X[1:150,:],Y[1:150],1e9);
pred=predict!(T,X)
Acc[counter]=correctrate(Y,convert(Array{Int,1},pred))

PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED
PAUSED


0.8546666666666667

In [1463]:
i=151
while i<1500
    counter = counter+1
    tic()
    expand!(T,X[i:i+149,:],Y[i:i+149,:],1e9);
    println(T)
    t=toq()
    println("The time is")
    println(t)
    push!(Time,t)
    pred=predict!(T,X)
    Acc[counter]=correctrate(Y,convert(Array{Int,1},pred))
    i=i+150
    println("The accuracy")
    println(Acc[counter])
    println("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
end

new extention
[72, 73, 5]
2
2.1
[0, 18, 0]
[72, 74, 5]
[0, 19, 0]
[72, 55, 5]
node done
[72, 55, 5]
[72, 55, 3]
[0, 0, 2]
node done
[72, 55, 3]
[15, 8, 2]
[57, 47, 1]
node done
[15, 8, 2]
[5, 3, 0]
[10, 5, 2]
node done
[5, 3, 0]
[1, 0, 0]
[4, 3, 0]
node done
[4, 3, 0]
[0, 1, 0]
[4, 2, 0]
node done
[4, 2, 0]
[3, 2, 0]
[1, 0, 0]
node done
[10, 5, 2]
[10, 5, 0]
[0, 0, 2]
node done
[10, 5, 0]
[9, 5, 0]
[1, 0, 0]
node done
[9, 5, 0]
[7, 5, 0]
[2, 0, 0]
node done
[7, 5, 0]
[7, 3, 0]
[0, 2, 0]
node done
[7, 3, 0]
[6, 3, 0]
[1, 0, 0]
node done
[6, 3, 0]
[3, 1, 0]
[3, 2, 0]
node done
[3, 1, 0]
[1, 0, 0]
[2, 1, 0]
node done
[57, 47, 1]
[50, 47, 0]
[7, 0, 1]
node done
[50, 47, 0]
[0, 18, 0]
[50, 29, 0]
node done
[50, 29, 0]
[33, 12, 0]
[17, 17, 0]
node done
[33, 12, 0]
[3, 0, 0]
[30, 12, 0]
node done
[30, 12, 0]
[8, 1, 0]
[22, 11, 0]
node done
[22, 11, 0]
[0, 10, 0]
[22, 1, 0]
node done
[22, 1, 0]
[16, 1, 0]
[6, 0, 0]
node done
[16, 1, 0]
[0, 1, 0]
[16, 0, 0]
node done
[17, 17, 0]
[14, 13, 0]
[3,

[74, 76, 6]
2
2.2
[74, 55, 6]
2
2.2
[0, 0, 2]
[74, 76, 7]
[0, 21, 0]
[74, 55, 7]
node done
[74, 55, 7]
[74, 55, 4]
[0, 0, 3]
node done
[74, 55, 4]
[15, 8, 2]
[59, 47, 2]
node done
[15, 8, 2]
[5, 3, 0]
[10, 5, 2]
node done
[5, 3, 0]
[1, 0, 0]
[4, 3, 0]
node done
[4, 3, 0]
[0, 1, 0]
[4, 2, 0]
node done
[4, 2, 0]
[3, 2, 0]
[1, 0, 0]
node done
[10, 5, 2]
[10, 5, 0]
[0, 0, 2]
node done
[10, 5, 0]
[9, 5, 0]
[1, 0, 0]
node done
[9, 5, 0]
[7, 5, 0]
[2, 0, 0]
node done
[7, 5, 0]
[7, 3, 0]
[0, 2, 0]
node done
[7, 3, 0]
[6, 3, 0]
[1, 0, 0]
node done
[6, 3, 0]
[3, 1, 0]
[3, 2, 0]
node done
[3, 1, 0]
[1, 0, 0]
[2, 1, 0]
node done
[59, 47, 2]
[52, 47, 0]
[7, 0, 2]
node done
[52, 47, 0]
[0, 18, 0]
[52, 29, 0]
node done
[52, 29, 0]
[34, 12, 0]
[18, 17, 0]
node done
[34, 12, 0]
[3, 0, 0]
[31, 12, 0]
node done
[31, 12, 0]
[8, 1, 0]
[23, 11, 0]
node done
[23, 11, 0]
[0, 10, 0]
[23, 1, 0]
node done
[23, 1, 0]
[17, 1, 0]
[6, 0, 0]
node done
[17, 1, 0]
[0, 1, 0]
[17, 0, 0]
node done
[18, 17, 0]
[15, 13, 0]


[4, 13, 0]
[2, 13, 0]
[2, 0, 0]
node done
[2, 13, 0]
[0, 4, 0]
[2, 9, 0]
node done
[2, 9, 0]
[0, 5, 0]
[2, 4, 0]
node done
[2, 4, 0]
[1, 3, 0]
[1, 1, 0]
node done
[1, 3, 0]
[0, 3, 0]
[1, 0, 0]
node done
[7, 0, 2]
[4, 0, 0]
[3, 0, 2]
node done
[3, 0, 2]
[2, 0, 2]
[1, 0, 0]
node done
[2, 0, 2]
[2, 0, 0]
[0, 0, 2]
node done
new extention
[76, 78, 7]
2
2.1
[0, 21, 0]
[76, 79, 7]
[0, 22, 0]
[76, 57, 7]
node done
[76, 57, 7]
[76, 57, 4]
[0, 0, 3]
node done
[76, 57, 4]
[16, 9, 2]
[60, 48, 2]
node done
[16, 9, 2]
[5, 3, 0]
[11, 6, 2]
node done
[5, 3, 0]
[1, 0, 0]
[4, 3, 0]
node done
[4, 3, 0]
[0, 1, 0]
[4, 2, 0]
node done
[4, 2, 0]
[3, 2, 0]
[1, 0, 0]
node done
[11, 6, 2]
[11, 6, 0]
[0, 0, 2]
node done
[11, 6, 0]
[10, 6, 0]
[1, 0, 0]
node done
[10, 6, 0]
[8, 6, 0]
[2, 0, 0]
node done
[8, 6, 0]
[8, 3, 0]
[0, 3, 0]
node done
[8, 3, 0]
[7, 3, 0]
[1, 0, 0]
node done
[7, 3, 0]
[4, 1, 0]
[3, 2, 0]
node done
[4, 1, 0]
[2, 0, 0]
[2, 1, 0]
node done
[60, 48, 2]
[53, 48, 0]
[7, 0, 2]
node done
[53, 48, 

[37, 13, 0]
[3, 0, 0]
[34, 13, 0]
node done
[34, 13, 0]
[8, 1, 0]
[26, 12, 0]
node done
[26, 12, 0]
[0, 11, 0]
[26, 1, 0]
node done
[26, 1, 0]
[20, 1, 0]
[6, 0, 0]
node done
[20, 1, 0]
[0, 1, 0]
[20, 0, 0]
node done
[20, 17, 0]
[17, 13, 0]
[3, 4, 0]
node done
[17, 13, 0]
[5, 13, 0]
[12, 0, 0]
node done
[5, 13, 0]
[2, 13, 0]
[3, 0, 0]
node done
[2, 13, 0]
[0, 4, 0]
[2, 9, 0]
node done
[2, 9, 0]
[0, 5, 0]
[2, 4, 0]
node done
[2, 4, 0]
[1, 3, 0]
[1, 1, 0]
node done
[1, 3, 0]
[0, 3, 0]
[1, 0, 0]
node done
[7, 0, 2]
[4, 0, 0]
[3, 0, 2]
node done
[3, 0, 2]
[2, 0, 2]
[1, 0, 0]
node done
[2, 0, 2]
[2, 0, 0]
[0, 0, 2]
node done
new extention
[80, 81, 7]
2
2.2
[80, 59, 7]
2
2.1
[80, 59, 4]
2
2.2
[64, 50, 2]
2
2.1
[57, 50, 0]
2
2.2
[57, 30, 0]
2
2.2
[20, 17, 0]
2
2.1
[17, 13, 0]
2
2.2
[12, 0, 0]
[81, 81, 7]
[0, 22, 0]
[81, 59, 7]
node done
[81, 59, 7]
[81, 59, 4]
[0, 0, 3]
node done
[81, 59, 4]
[16, 9, 2]
[65, 50, 2]
node done
[16, 9, 2]
[5, 3, 0]
[11, 6, 2]
node done
[5, 3, 0]
[1, 0, 0]
[4, 3, 0

[2, 4, 0]
[1, 3, 0]
[1, 1, 0]
node done
[1, 3, 0]
[0, 3, 0]
[1, 0, 0]
node done
[7, 0, 2]
[4, 0, 0]
[3, 0, 2]
node done
[3, 0, 2]
[2, 0, 2]
[1, 0, 0]
node done
[2, 0, 2]
[2, 0, 0]
[0, 0, 2]
node done
new extention
[82, 85, 7]
2
2.2
[82, 61, 7]
2
2.1
[82, 61, 4]
2
2.2
[66, 52, 2]
2
2.1
[59, 52, 0]
2
2.2
[59, 31, 0]
2
2.1
[38, 14, 0]
2
2.2
[35, 14, 0]
2
2.2
[27, 13, 0]
2
2.2
[27, 1, 0]
2
2.1
[21, 1, 0]
2
2.2
[21, 0, 0]
[83, 85, 7]
[0, 24, 0]
[83, 61, 7]
node done
[83, 61, 7]
[83, 61, 4]
[0, 0, 3]
node done
[83, 61, 4]
[16, 9, 2]
[67, 52, 2]
node done
[16, 9, 2]
[5, 3, 0]
[11, 6, 2]
node done
[5, 3, 0]
[1, 0, 0]
[4, 3, 0]
node done
[4, 3, 0]
[0, 1, 0]
[4, 2, 0]
node done
[4, 2, 0]
[3, 2, 0]
[1, 0, 0]
node done
[11, 6, 2]
[11, 6, 0]
[0, 0, 2]
node done
[11, 6, 0]
[10, 6, 0]
[1, 0, 0]
node done
[10, 6, 0]
[8, 6, 0]
[2, 0, 0]
node done
[8, 6, 0]
[8, 3, 0]
[0, 3, 0]
node done
[8, 3, 0]
[7, 3, 0]
[1, 0, 0]
node done
[7, 3, 0]
[4, 1, 0]
[3, 2, 0]
node done
[4, 1, 0]
[2, 0, 0]
[2, 1, 0]
node don

...

[29, 10, 3]
[29, 10, 0]
[0, 0, 3]
node done
[29, 10, 0]
[24, 10, 0]
[5, 0, 0]
node done
[24, 10, 0]
[10, 10, 0]
[14, 0, 0]
node done
[10, 10, 0]
[10, 3, 0]
[0, 7, 0]
node done
[10, 3, 0]
[9, 3, 0]
[1, 0, 0]
node done
[9, 3, 0]
[6, 1, 0]
[3, 2, 0]
node done
[6, 1, 0]
[4, 0, 0]
[2, 1, 0]
node done
[141, 116, 5]
[125, 116, 0]
[16, 0, 5]
node done
[125, 116, 0]
[0, 56, 0]
[125, 60, 0]
node done
[125, 60, 0]
[80, 26, 0]
[45, 34, 0]
node done
[80, 26, 0]
[15, 0, 0]
[65, 26, 0]
node done
[65, 26, 0]
[8, 1, 0]
[57, 25, 0]
node done
[57, 25, 0]
[0, 24, 0]
[57, 1, 0]
node done
[57, 1, 0]
[47, 1, 0]
[10, 0, 0]
node done
[47, 1, 0]
[0, 1, 0]
[47, 0, 0]
node done
[45, 34, 0]
[42, 30, 0]
[3, 4, 0]
node done
[42, 30, 0]
[16, 30, 0]
[26, 0, 0]
node done
[16, 30, 0]
[2, 30, 0]
[14, 0, 0]
node done
[2, 30, 0]
[0, 12, 0]
[2, 18, 0]
node done
[2, 18, 0]
[0, 10, 0]
[2, 8, 0]
node done
[2, 8, 0]
[1, 7, 0]
[1, 1, 0]
node done
[1, 7, 0]
[0, 7, 0]
[1, 0, 0]
node done
[16, 0, 5]
[11, 0, 0]
[5, 0, 5]
node done
[

[16, 0, 5]
[11, 0, 0]
[5, 0, 5]
node done
[5, 0, 5]
[4, 0, 5]
[1, 0, 0]
node done
[4, 0, 5]
[4, 0, 0]
[0, 0, 5]
node done
new extention
[185, 199, 19]
2
2.2
[185, 130, 19]
2
2.1
[185, 130, 8]
2
2.1
[41, 14, 3]
2
2.2
[30, 10, 3]
2
2.1
[30, 10, 0]
2
2.1
[25, 10, 0]
2
2.1
[11, 10, 0]
2
2.1
[11, 3, 0]
2
2.2
[1, 0, 0]
[186, 199, 19]
[0, 69, 0]
[186, 130, 19]
node done
[186, 130, 19]
[186, 130, 8]
[0, 0, 11]
node done
[186, 130, 8]
[42, 14, 3]
[144, 116, 5]
node done
[42, 14, 3]
[11, 4, 0]
[31, 10, 3]
node done
[11, 4, 0]
[4, 0, 0]
[7, 4, 0]
node done
[7, 4, 0]
[0, 2, 0]
[7, 2, 0]
node done
[7, 2, 0]
[3, 2, 0]
[4, 0, 0]
node done
[31, 10, 3]
[31, 10, 0]
[0, 0, 3]
node done
[31, 10, 0]
[26, 10, 0]
[5, 0, 0]
node done
[26, 10, 0]
[12, 10, 0]
[14, 0, 0]
node done
[12, 10, 0]
[12, 3, 0]
[0, 7, 0]
node done
[12, 3, 0]
[10, 3, 0]
[2, 0, 0]
node done
[10, 3, 0]
[7, 1, 0]
[3, 2, 0]
node done
[7, 1, 0]
[5, 0, 0]
[2, 1, 0]
node done
[144, 116, 5]
[128, 116, 0]
[16, 0, 5]
node done
[128, 116, 0]
[0, 56

[2, 18, 0]
2
2.2
[2, 8, 0]
2
2.1
[1, 7, 0]
2
2.1
[0, 7, 0]
[188, 202, 19]
[0, 70, 0]
[188, 132, 19]
node done
[188, 132, 19]
[188, 132, 8]
[0, 0, 11]
node done
[188, 132, 8]
[43, 14, 3]
[145, 118, 5]
node done
[43, 14, 3]
[11, 4, 0]
[32, 10, 3]
node done
[11, 4, 0]
[4, 0, 0]
[7, 4, 0]
node done
[7, 4, 0]
[0, 2, 0]
[7, 2, 0]
node done
[7, 2, 0]
[3, 2, 0]
[4, 0, 0]
node done
[32, 10, 3]
[32, 10, 0]
[0, 0, 3]
node done
[32, 10, 0]
[27, 10, 0]
[5, 0, 0]
node done
[27, 10, 0]
[12, 10, 0]
[15, 0, 0]
node done
[12, 10, 0]
[12, 3, 0]
[0, 7, 0]
node done
[12, 3, 0]
[10, 3, 0]
[2, 0, 0]
node done
[10, 3, 0]
[7, 1, 0]
[3, 2, 0]
node done
[7, 1, 0]
[5, 0, 0]
[2, 1, 0]
node done
[145, 118, 5]
[129, 118, 0]
[16, 0, 5]
node done
[129, 118, 0]
[0, 56, 0]
[129, 62, 0]
node done
[129, 62, 0]
[81, 27, 0]
[48, 35, 0]
node done
[81, 27, 0]
[15, 0, 0]
[66, 27, 0]
node done
[66, 27, 0]
[8, 1, 0]
[58, 26, 0]
node done
[58, 26, 0]
[0, 24, 0]
[58, 2, 0]
node done
[58, 2, 0]
[48, 2, 0]
[10, 0, 0]
node done
[48, 

Excessive output truncated after 524288 bytes.

[0, 57, 0]
[129, 63, 0]
node done
[129, 63, 0]
[81, 28, 0]
[48, 35, 0]
node done
[81, 28, 0]
[15, 0, 0]
[66, 28, 0]
node done
[66, 28, 0]
[8, 1, 0]
[58, 27, 0]
node done
[58, 27, 0]
[0, 25, 0]
[58, 2, 0]
node done
[58, 2, 0]
[48, 2, 0]
[10, 0, 0]
node done
[48, 2, 0]
[0, 2, 0]
[48, 0, 0]
node done
[48, 35, 0]
[45, 31, 0]
[3, 4, 0]
node done
[45, 31, 0]
[18, 31, 0]
[27, 0, 0]
node done
[18, 31, 0]
[2, 31, 0]
[16, 0, 0]
node done
[2, 31, 0]
[0, 12, 0]
[2, 19, 0]
node done
[2, 19, 0]
[0, 10, 0]
[2, 9, 0]
node done
[2, 9, 0]
[1, 8, 0]
[1, 1, 0]
node done
[1, 8, 0]
[0, 8, 0]
[1, 0, 0]
node done
[16, 0, 5]
[11, 0, 0]
[5, 0, 5]
node done
[5, 0, 5]
[4, 0, 5]
[1, 0, 0]
node done
[4, 0, 5]
[4, 0, 0]
[0, 0, 5]
node done
new extention
[188, 205, 19]
2
2.2
[188, 135, 19]
2
2.1
[188, 135, 8]
2
2.2
[145, 120, 5]
2
2.1
[129, 120, 0]
2
2.2
[129, 63, 0]
2
2.2
[48, 35, 0]
2
2.1
[45, 31, 0]
2
2.2
[27, 0, 0]
[189, 205, 19]
[0, 70, 0]
[189, 135, 19]
node done
[189, 135, 19]
[189, 135, 8]
[0, 0, 11]
node don

In [1464]:
Acc

10-element Array{Float64,1}:
 0.854667
 0.854667
 0.854667
 0.854667
 0.854667
 0.854667
 0.854667
 0.772   
 0.772   
 0.772   

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [ ]:
MF=Mondrian_Forest_Classifier(100)
Acc=zeros(150);
Time=[];

In [ ]:
counter=1
tic()
train!(MF,X[1:150,:],Y[1:150],1e9);
toc()
pred=predict!(MF,x)
Acc[counter]=correctrate(y,convert(Array{Int,1},pred))

In [ ]:
tic()
expand_forest!(MF,X[151:300,:],Y[151:300,:],1e9)
toc()

In [ ]:
pred=predict!(MF,x);

In [ ]:
correctrate(y,convert(Array{Int,1},pred))

In [ ]:
expand_forest!(MF,X[151:300,:],Y[151:300,:],1e9);
pred=predict!(MF,x)
Acc[counter]=correctrate(y,convert(Array{Int,1},pred))

In [ ]:
i=151
while i<15000
    counter = counter+1
    tic()
    expand_forest!(MF,X[i:i+149,:],Y[i:i+149,:],1e9);
    t=toq()
    println("The time is")
    println(t)
    push!(Time,t)
    pred=predict!(MF,x)
    Acc[counter]=correctrate(y,convert(Array{Int,1},pred))
    i=i+150
    println("The accrucary")
    println(Acc[counter])
end

In [7]:
include("Mondrian_extention_utils.jl");
"""
`function Extend_Mondrian_Tree!(T::Mondrian_Tree,λ::Float64,X::Array{Float64} where N,Y::Int64) `

This function extends an already existing Mondrian Tree by ONE new datapoint that gets incorperated in the tree. 

`Input`: Mondrian Tree T (abstract type Mondrian_Tree), Lifetime parameter λ (Float64), 1dim Array of Features Array X (Array of Float64), classlabel Y (Int64)

`Output`: Mondrian Tree

`Files needed to run this function`: "Mondrian_Forest_Classifier.jl", "Mondrian_extention.jl"

This function calls the function Extend_Mondrian_Block.

The usage of the function "expand!" is recommended to expand Mondrian Trees as it has a nicer user interface.

"""
## Algorithm 3 in the Paper "Mondrian Forests: Efficient Online Random Forests"
function Extend_Mondrian_Tree!(T::Mondrian_Tree,λ::Float64,X::Array{Float64},Y::Int64) 
    println("new extention")
    ϵ=get(T.root)
    Extend_Mondrian_Block!(T,λ,ϵ,X,Y)
    j_print(T.root)
    return T
end


"""
`function Extend_Mondrian_Block!(T::Mondrian_Tree,λ::Float64,j::Mondrian_Node,X::Array{Float64},Y::Int64)`

This function extends a Mondrian Block to incorporate ONE new datapoint. 

`Input`: Mondrian Tree T (abstract type Mondrian_Tree), Lifetime parameter λ (Float64), Mondrian Node j (abstract type Mondrian_Node), 1dim Array of Features Array X (Array of Float64), classlabel Y (Int64)

`Output`: Modified nodes in the input tree

`Files needed to run this function`: Mondrian_Forest_Classifier.jl", "Mondrian_extention.jl"

This function calls the functions 

- Extended_dimension()

- sample_extended_split_dimension

- Sample_Mondrian_Block!

- update_intervals


"""

## Algorithm 4 in the Paper 
function Extend_Mondrian_Block!(T::Mondrian_Tree,λ::Float64,j::Mondrian_Node,X::Array{Float64},Y::Int64)
    println(j.c)
if sum(j.c .> 0) == 1  #check if all labels are identical
     Θ = update_intervals(get(j.Θ),X)        # update extent
     j.Θ=Θ 
         if findmax(j.c)[2] == Y
             i = findmax(j.c)[2]
             j.c[i] = j.c[i]+1
             backpropergate_c_tab(j,Y)
             return
         else
            j.node_type = [true,false,false]
            A=zeros(1,length(X))
            A[:,:]=X
            Sample_Mondrian_Block!(j,get(j.Θ),λ,T,A,[Y])
         end        
        
else
       #println("not a leaf")
    E = rand(Exponential(1/Extended_dimension(get(j.Θ),X)))  #sample value E
    if j.node_type[3]==true
        # check if the node we're looking at is the root (if yes the split time is assumed to be 0)
            #println("j is the root")
        τₚ = 0
    else
            #println("j is not the root")
        τₚ = (get(j.parent)).τ                               # if it's not the root get the split time of the node above j
    end
    if τₚ + E < j.τ                                          # check if our split happens in time
          println(1)                                                  
        d,x= sample_extended_split_dimension(get(j.Θ),X)     # sample new split dimension / split direction
        Θ = update_intervals(get(j.Θ),X)                     # get the boxes for the new node
        if j.node_type[3]==true                              # check if we replace the root
            j_wave=Mondrian_Node(E,[false,false,true])       # replace the root by the new node j_wave
            j.node_type = [true,false,false]
        else
            j_wave=Mondrian_Node(get(j.parent).τ+E,[true,false,false])  #if we don't replace the root, introcue a new node j_wave, parent to j
            j_wave.parent = j.parent
            if j == get(j.parent).left             # check if j was left or right child of j_parent
                get(j.parent).left = j_wave
            else
                get(j.parent).right = j_wave
            end
        end
        j_wave.δ = d
        j_wave.ζ = x
        j_wave.Θ = Θ 
        update_counts_extended(j_wave,j,Y)
        j.parent=j_wave
        backpropergate_c_tab(j_wave,Y) 
            
        j_prime = Mondrian_Node(0.0, [true,false,false])   #initialise new sibling to j
        j_prime.parent = j_wave
        j_prime.tab = zeros(size(j_wave.tab))
        j_prime.tab[Y] = 1
        j_prime.c = zeros(size(j_wave.c))
        j_prime.c[Y] = 1
        j_prime.Gₚ=zeros(size(j_wave.Gₚ))
        
        if X[d] > x     # check where the new datapoint lies
            j_wave.left = j
            j_wave.right = j_prime 
                if get(j.Θ).Intervals[d,2]> x    
                    get(j.Θ).Intervals[d,2] = x  #adapt box of j according to new split
                end
            j_prime.Θ = j_wave.Θ
            get(j_wave.Θ).Intervals[d,1] = x
            A=zeros(1,length(X))
            A[:,:]=X
            Sample_Mondrian_Block!(j_prime,get(j_prime.Θ),λ,T,A,[Y])  #sample a mondrian block at the node whose associated boxes contain the new datapoint
        else
            j_wave.left = j_prime
            j_wave.right = j
            if get(j.Θ).Intervals[d,1]< x 
                get(j.Θ).Intervals[d,1]= x      #adapt box of j according to new split
            end
            j_prime.Θ = j_wave.Θ
            get(j_wave.Θ).Intervals[d,2] = x
            A=zeros(1,length(X))
            A[:,:]=X
            Sample_Mondrian_Block!(j_prime,get(j_prime.Θ),λ,T,A,[Y])  #sample a mondrian block at the node whose associated boxes contain the new datapoint
        end


    else
              println(2)                                  # if the split didn't occur in time
        Θ = update_intervals(get(j.Θ),X)        # update the boxes of j
        j.Θ=Θ
        if j.node_type != [false,true,false]    # check if j is a leaf
            if X[get(j.δ)] < get(j.ζ)           # if the new datapoint is in the boxes associated with the left child of j -> Extend towards the left child, else the right
                     println(2.1)
                Extend_Mondrian_Block!(T,λ,get(j.left),X,Y)
            else
                    println(2.2)
                Extend_Mondrian_Block!(T,λ,get(j.right),X,Y)
                
            end
        end
    end
end
end

"""
`function expand!(T::Mondrian_Tree_Classifier,X::Array{Float64,N} where N,Y::Array{Int64},λ::Float64)`

This function expands an already sampled Mondrian Tree Classifier by a desired number of datapoints. 

`Input`: Mondrian Tree Classifier T (abstract type Mondrian_Tree_Classifier), array of features X (Array of Float64), array of class labels (1dim of Float 64), Lifetime parameter λ (Float 64)

Each row in the array X represents one set of features, the corresponding row in Y represents the class label. 

`Output`: Mondrian Tree Classifier with incoporated new datapoints

`Files needed to run this function`: Mondrian_Forest_Classifier.jl", "Mondrian_extention.jl"

This function calls the function Extend_Mondrian_Tree. 
"""

function expand!(T::Mondrian_Tree_Classifier,X::Array{Float64,N} where N,Y::Array{Int64},λ::Float64)    
    
    # puts the extention in a nice framework, allows to extend by multiple datapoints
    
    for i=1:length(X[:,1])
        T.Tree=Extend_Mondrian_Tree!(T.Tree,λ,X[i,:],Y[i]);
    end
    T.X = vcat(T.X,X)
    T.Y = vcat(T.Y,Y)
    compute_predictive_posterior_distribution!(T.Tree,10*size(X,2))
    return T    
end

"""
`function expand_forest!(MF::Mondrian_Forest_Classifier,X_extend, Y_extend,λ)`

This function expands an already sampled Mondrian Forest Classifier by a desired number of datapoints.

`Input`: Mondrian Forest Classifier MF (abstract type Mondrian_Forest_Classifier), array of features X_extend to extend the forest on, Array of class labels Y_extend corresponding to the new features, life time parameter λ

`Output`: Mondrian Forest Classifier with incoporated new datapoints

`Files needed to run this function`: Mondrian_Forest_Classifier.jl", "Mondrian_extention.jl"

This function calls the function expand!. 
"""

function expand_forest!(MF::Mondrian_Forest_Classifier,X_extend, Y_extend,λ)
    X=MF.X
    Y=MF.Y
    if size(X)[2] != size(X_extend)[2]
        println("Error - the number of features in the new data doesn't fit the original data")
    end
    Trees=MF.Trees
    @parallel for i=1:MF.n_trees
        T = expand!(Trees[i], X_extend,Y_extend,λ)
        Trees[i]=T
    end
     MF.Trees=Trees
     MF.X=vcat(X,X_extend)
     MF.Y=vcat(Y,Y_extend)
    
end

expand_forest!

In [6]:
function update_counts_extended(j_wave,j,Y)
    j_wave.tab = copy(j.tab)
    j_wave.c = copy(j.c)    
    j_wave.Gₚ = zeros(size(j.Gₚ))    
    if j_wave.tab[Y] == 0 
    j_wave.tab[Y] = 1
    end     
    j_wave.c[Y] = j_wave.c[Y]+1
end

update_counts_extended (generic function with 1 method)

In [5]:
function backpropergate_c_tab(j_wave,Y)
    if j_wave.node_type != [false, false, true]
   get(j_wave.parent).c[Y] =  get(j_wave.parent).c[Y]+1
    if get(j_wave.parent).tab[Y] == 0 
    get(j_wave.parent).tab[Y] = 1
    end
    if get(j_wave.parent).node_type != [false,false,true]
        backpropergate_c_tab(get(j_wave.parent),Y)
    end
    end
end

backpropergate_c_tab (generic function with 1 method)

In [4]:
function leaf_check(j)
   if get(j).node_type == [false, true, false]
        println(get(j).c)
    else
        leaf_check(get(j).left)
        leaf_check(get(j).right)
    end
end

leaf_check (generic function with 1 method)

In [3]:
function j_print(j)
   if get(j).node_type != [false, true, false]
        println(get(j).c)
        println(get(get(j).left).c)
        println(get(get(j).right).c)
        println("node done")
        j_print(get(j).left)
        j_print(get(j).right)
    end
    
end

j_print (generic function with 1 method)